In [1]:
import pandas as pd
import numpy as np
import re
import pysam
from Bio import Seq

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [2]:
# load weigthed RSCU Score
ws_df = pd.read_csv("codon_adaptive_ws.txt",sep="\t",index_col=0)
ws_df.head(2)

,cai_ln_ws,cui_ln_ws
TGT,-0.624154,-0.33417
TGC,0.000000,0.00000


In [3]:
# load coding sequences
filename = 'mRNA1273.fasta'
read_fa = pysam.FastaFile(filename)
primary_cds = read_fa.fetch(read_fa.references[0])

In [4]:
# optimal codon
optimal_codon = {}
optimal_score = {}
nonoptimal_codon = {}
nonoptimal_score = {}

for index, row in ws_df.iterrows():
    
    protein = str(Seq.Seq(index).translate())
    score = row['cui_ln_ws']
    codon = index
    
    if protein in optimal_codon:
        if score > optimal_score[protein]:
            optimal_codon[protein] = codon
            optimal_score[protein] = score
        if score < nonoptimal_score[protein]:
            nonoptimal_codon[protein] = codon
            nonoptimal_score[protein] = score
    else:
        optimal_codon[protein] = codon
        optimal_score[protein] = score
        nonoptimal_codon[protein] = codon
        nonoptimal_score[protein] = score
        
# add M, ATG
optimal_codon['M'] = 'ATG'
optimal_codon['*'] = 'TAA'

In [5]:
# optimal sequence generate
protein_seq = str(Seq.Seq(primary_cds).translate())

codons_optimal = []
for i in range(0, len(protein_seq), 1):
    codons_optimal.append(optimal_codon[protein_seq[i]])
    
optimal_cds = ''.join(codons_optimal)

In [6]:
print(">optimal_seq")
print(optimal_cds)

>optimal_seq
ATGTTCGTGTTCCTGGTGCTGCTGCCCCTGGTGAGCAGCCAGTGCGTGAACCTGACCACCCGGACCCAGCTGCCCCCCGCCTACACCAACAGCTTCACCCGGGGCGTGTACTACCCCGACAAGGTGTTCCGGAGCAGCGTGCTGCACAGCACCCAGGACCTGTTCCTGCCCTTCTTCAGCAACGTGACCTGGTTCCACGCCATCCACGTGAGCGGCACCAACGGCACCAAGCGGTTCGACAACCCCGTGCTGCCCTTCAACGACGGCGTGTACTTCGCCAGCACCGAGAAGAGCAACATCATCCGGGGCTGGATCTTCGGCACCACCCTGGACAGCAAGACCCAGAGCCTGCTGATCGTGAACAACGCCACCAACGTGGTGATCAAGGTGTGCGAGTTCCAGTTCTGCAACGACCCCTTCCTGGGCGTGTACTACCACAAGAACAACAAGAGCTGGATGGAGAGCGAGTTCCGGGTGTACAGCAGCGCCAACAACTGCACCTTCGAGTACGTGAGCCAGCCCTTCCTGATGGACCTGGAGGGCAAGCAGGGCAACTTCAAGAACCTGCGGGAGTTCGTGTTCAAGAACATCGACGGCTACTTCAAGATCTACAGCAAGCACACCCCCATCAACCTGGTGCGGGACCTGCCCCAGGGCTTCAGCGCCCTGGAGCCCCTGGTGGACCTGCCCATCGGCATCAACATCACCCGGTTCCAGACCCTGCTGGCCCTGCACCGGAGCTACCTGACCCCCGGCGACAGCAGCAGCGGCTGGACCGCCGGCGCCGCCGCCTACTACGTGGGCTACCTGCAGCCCCGGACCTTCCTGCTGAAGTACAACGAGAACGGCACCATCACCGACGCCGTGGACTGCGCCCTGGACCCCCTGAGCGAGACCAAGTGCACCCTGAAGAGCTTCACCGTGGAGAAGGGCATCTACCAGACCAGCAACTTCCGGGTGCAGCCCACCGAGAGCATCGTGCGGTTC

In [7]:
def _cai_seq(seq, ws):
    """Calculate the CAI (float) for the provided cds sequence (string).
    This method uses the Index and returns the CAI for the given sequence.
    """
    seq = seq.upper()

    codon_id, codon_count = np.unique(re.findall('...', seq),return_counts=True)
    codon_count = pd.Series(codon_count, index=codon_id, name='codons')
    
    # remove start or stop codons
    df = pd.concat([ws, codon_count], axis=1, sort=True).loc[ws.index]
    
    # calculate cai
    codon_prop = df['codons'] / df['codons'].sum()
    return np.exp(np.sum(df.iloc[:,0] * codon_prop))

In [8]:
print("CAI of primary coding sequence:")
print(_cai_seq(primary_cds,ws_df['cui_ln_ws']))
print("CAI of optimized coding sequence:")
print(_cai_seq(optimal_cds,ws_df['cui_ln_ws']))

CAI of primary coding sequence:
0.9720393729996601
CAI of optimized coding sequence:
1.0
